In [1]:
!pip install -q langchain
!pip install -q langchain-nvidia-ai-endpoints
!pip install -q langchain-community langchain-core
!pip install -q --upgrade langchain
!pip install -q pandas

In [22]:
from langchain.schema import SystemMessage, HumanMessage
from langchain_nvidia_ai_endpoints import ChatNVIDIA

llm = ChatNVIDIA(
    model="openai/gpt-oss-20b",
    base_url="http://localhost:8000/v1",
    max_tokens=2048,
    temperature=0.7,
)

messages = [
    # Change this to on if you want to use reasoning
    SystemMessage(content="detailed thinking off"),
    HumanMessage(content="Write a limerick about the wonders of GPU computing.")
]

response = llm(messages)
print(response.content)

/tmp/ipykernel_184886/402167311.py:4: DeprecationWarning: The 'max_tokens' parameter is deprecated and will be removed in a future version. Please use 'max_completion_tokens' instead.
  llm = ChatNVIDIA(


There once was a chip named GPU,  
Whose cores ran in parallel crew.  
It crunched data so fast,  
From graphics to AI,  
Turning science into bright, blazing hue.


In [12]:
print(response.additional_kwargs['reasoning_content'])  # if reasoning was enabled

The user requests: "Write a limerick about the wonders of GPU computing." We need to respond with a limerick. A limerick is a humorous poem of five lines, with an AABBA rhyme scheme and typically anapestic meter. We can produce a creative limerick about GPU computing. Ensure line breaks and rhyme scheme. Use rhyme pattern: AABB A? Actually typical limerick: lines 1,2,5 rhyme; lines 3,4 rhyme. So we need to produce something like: "There once was a coder named Sue, ..."

We need to produce a limerick about GPU computing wonders. So mention GPUs, parallelism, speed, crunching, etc. So produce a limerick with witty humor maybe. Ensure it's well-structured. We'll do it: 5 lines, with rhyme. Use humor. Something like:

"There once was a coder named Lou,
Who found GPUs faster than a kangaroo.
With threads all aligned,
It left circuits behind,
Now his models compute in a jiffy, too!"

But need to rhyme lines 1,2,5: maybe "Lou", "kangaroo", "jiffy" no. Let's craft properly:

Line 1: "There onc

In [3]:
event_types_mapping = {"analyst rating": 0, "price targets": 1,
                       "earnings": 2, "labour related": 3, "mergers and acquisitions": 4, "dividends": 5,
                       "regulatory": 6, "stock price movement": 7, "credit ratings": 8, "products-services": 9,
                       "product approval": 10, "guidance": 11, "other": 12
                      }

In [4]:
prompt = """
You are a helpful AI assistant that analyses financial news headlines and identifies what event type is described.
You will classify event types into one of the following categories (in square brackets)

- [Analyst Rating]: An entity such as a bank, asset manager, etc. gives a classification/rating/downgrade/upgrade/opinion to an asset.
                    If there is no specified analyst and company given, it's not Analyst Rating and should be classified as OTHER.
- [Price Targets]: A mention of a price target (PT) is given by an entity such as bank, asset manager, etc.
                   This takes priority over any other class, so if a price target is present use this class!
- [Earnings]: Reports of quarterly, monthly, etc. concrete values of revenue, ESP, etc. Percentage fluctations.
              Expected values are not Earnings and should be Guidance instead!
- [Labour Issues]: Mentions of layoffs, union action, strikes, rising cost of labour, bonuses for execs, etc. Important personal change, e.g. CEO, CFO, VPs, etc.
- [Mergers and Acquisitions]: Whenever merging or acquisition of entities, not just companies, is mentioned.
                              Things like partnerships do not belong to this class! Takes priority over other classes.
- [Dividends]: Mentions of dividend performance, dividend per share, decisions not to issue dividends, etc.
- [Regulatory]: Mentions corporate position focused on environmental affairs, government regulation, international treaties, geopolitics, debt repayment, licenses, patents, etc.
                Any executive decisions taken by the government are Regulatory. Takes priority over other classes.
- [Stock price movement]: Includes pricing of public offerings, daily, monthly and yearly movements, highs and lows, options trade and alerts, etc.
                          Only relevant when a specific entity/industry or set of entities/industries mentioned.
                          Quantities such as sales, are not stock price movements. Stock splits do not count as stock movements.
                          Takes priority over Earnings, Stock Price Movement, Credit Ratings
- [Credit Ratings]: Adjustments of company's borrowing capacity, changes in debt values, changes in ratings, etc.
- [Products-Services]: Mentions of a company's particular product, forward-looking product directions, disruption, government and private contracts, etc.
                       Any sort of delay regarding a product roll-out, etc.
- [Product Approval]: Mentions FDA approvals, environmental approvals, acceptance for review. Any time an entity approves the rollout of a corporations product.
- [Guidance]: Forward looking statements issued by the company's themselves regarding Revenue, EPS, potential sales going up/down, number of contracts, etc.
              Similar talk to earnings but about projections rather than realised.
              
If the headline doesn't match any of the classes, classify it as OTHER.
ATTENTION:
  - Only assign a category if the headline meets all the criteria listed for that category. Otherwise use OTHER
  - Encourage precise matching rather than assigning categories based on partial or superficial similarities
  - OTHER is the default category when in doubt
  - If there are no specific companies mentioned, use OTHER

A few examples:

1. Cornerstone OnDemand Higher as Barclays Upgraded to Overweight -> [Analyst Rating]
2. HC Wainwright & Co. Maintains Buy on Balchem, Lowers Price Target to $104 -> [Price Targets]
3. Sina Reports Q4 Adj. EPS $0.24 vs $0.18 Est., Sales $211.1M vs $207.6M Est.; Sees FY15 Sales $800M-$900M vs $884.6M Est. -> [Earnings]
4. Delta To Buy Out Employees, Offer Early Retirement (DAL) -> [Labour Issues]
5. Constant Contact Announces Deal to Be Purchased by Endurance Int'l at $32/Share -> [Mergers and Acquisitions]
6. Ameriprise Financial Announces 12% Qtr. Dividend Increase to $0.58/Share -> [Dividends]
7. Ultratech Achieves ISO 9001 and 14001 Certification for Singapore Operations and Recertification for U.S. Facility  -> [Regulatory]
8. Mid-Afternoon Market Update: Dow Up Over 200 Points; Lakeland Industries Shares Spike Higher -> [Stock price movement]
9. Moody's Affirms Ratings on Nokia; Outlook Revised from Negative to Developing -> [Credit Ratings]
10. NOVAVAX Awarded HHS-BARDA Contract Valued at up to $179 Million  -> [Products-Services]
11. Sanofi's Genzyme Announces Lemtrada Resubmission Accepted for Review by FDA  -> [Product Approval]
12. Dot Hill Systems Announces Re-alignment of Software Development Initiatives; Sees Q4 EPS of $(0.02)-(0.03) vs $0.03-(0.03) Prior  -> [Guidance]
13. Kopin Chairman Fan Buys 116,400 Shares @$2.83/Share -Form 4 -> [OTHER]

Given the following headline:

### START HEADLINE ###

{headline}

### END HEADLINE ###

What event type best classifies it? Answer only with your predicted class and give it inside double square brackets, like [[class]]
"""

In [5]:
import pickle
from collections import Counter


with open('/home/shadeform/nemo_customiser_k8_utils/manually_labelled_testset.pkl', 'rb') as f:
    test_labels = pickle.load(f)

Counter(list(test_labels.values()))

Counter({'Analyst Rating': 96,
         'OTHER': 93,
         'Mergers and Acquisitions': 88,
         'Labour Issues': 88,
         'Regulatory': 85,
         'Guidance': 83,
         'Stock price movement': 76,
         'Earnings': 75,
         'Product Approval': 72,
         'Dividends': 71,
         'Price Targets': 66,
         'Credit Ratings': 61,
         'Products-Services': 49,
         'Management changes': 22})

In [6]:
import nest_asyncio
nest_asyncio.apply()

In [44]:
import asyncio
import threading


completed_tasks = 0  # Variable to track completed tasks
texts = [text for text in test_labels]
total_tasks = len(texts)
lock = asyncio.Lock()  # To protect shared variables during concurrent access
predictions = {}


def run(text):
    response = llm.invoke(prompt.format(headline=text))
    return response.additional_kwargs['reasoning_content']


def save_item(item_to_save, file_name):
    with open(file_name, 'wb') as f:
        pickle.dump(item_to_save, f)
        

async def run_with_timeout(text, semaphore):
    global completed_tasks
    attempts = 0
    while True:
        attempts += 1
        try:
            async with semaphore:
                # Run the synchronous `run` function in a thread with a timeout
                result = await asyncio.wait_for(asyncio.to_thread(run, text), timeout=30)
            predictions[text] = result
            async with lock:
                completed_tasks += 1
                if completed_tasks % 200 == 0:
                    print(f"Completed {completed_tasks}/{total_tasks} tasks.")
            break  # Exit loop on success
        except asyncio.TimeoutError:
            print(f"Task for '{text}' timed out. Retrying (attempt {attempts})...")
            continue
        except Exception as e:
            # Handle other exceptions if necessary
            print(f"Task for '{text}' failed with exception {e}. Retrying (attempt {attempts})...")
            continue

async def main():
    semaphore = asyncio.Semaphore(5)
    tasks = [asyncio.create_task(run_with_timeout(text, semaphore)) for text in texts]
    await asyncio.gather(*tasks)

asyncio.run(main())

Completed 200/1025 tasks.
Completed 400/1025 tasks.
Completed 600/1025 tasks.
Completed 800/1025 tasks.
Completed 1000/1025 tasks.


In [45]:
import re

def extract_classification(text):
    """Extract the classification from text between [[ ]]"""
    if not text:
        return None
    pattern = r'\[\[(.*?)\]\]'
    match = re.search(pattern, text)
    if match:
        return match.group(1).strip()
    return None

# Extract classifications from predictions
extracted_predictions = {}
y_pred = []
y_true = []
for headline, response_text in predictions.items():
    classification = extract_classification(response_text)
    if classification is None:
        classification = "Unknown"
    extracted_predictions[headline] = classification
    y_pred.append(classification)
    y_true.append(test_labels[headline])



# Show some statistics
total_predictions = len(extracted_predictions)
successful_extractions = sum(1 for v in extracted_predictions.values() if v is not None)
print(f"\nTotal predictions: {total_predictions}")
print(f"Successful extractions: {successful_extractions}")
print(f"Failed extractions: {total_predictions - successful_extractions}")


Total predictions: 1025
Successful extractions: 1025
Failed extractions: 0


In [46]:
import numpy as np

np.mean(np.array(y_true) == np.array(y_pred))

np.float64(0.791219512195122)

In [47]:
from sklearn.metrics import f1_score

# Should be 0.914 for straight classification
f1 = f1_score(y_true, y_pred, average='macro')
f1

0.6674324156937987

In [48]:
f1 = f1_score(y_true, y_pred, average=None)

for idx, k in enumerate(event_types_mapping):
    print(k)
    print(f1[idx])

analyst rating
0.8865979381443299
price targets
0.95
earnings
0.9635036496350365
labour related
0.8811188811188811
mergers and acquisitions
0.8947368421052632
dividends
0.6875
regulatory
0.0
stock price movement
0.8823529411764706
credit ratings
0.4774774774774775
products-services
1.0
product approval
0.819672131147541
guidance
0.7843137254901961
other
0.7058823529411765


In [49]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

                          precision    recall  f1-score   support

          Analyst Rating       0.88      0.90      0.89        96
          Credit Ratings       0.97      0.93      0.95        61
               Dividends       1.00      0.93      0.96        71
                Earnings       0.93      0.84      0.88        75
                Guidance       0.99      0.82      0.89        83
           Labour Issues       0.76      0.62      0.69        88
      Management changes       0.00      0.00      0.00        22
Mergers and Acquisitions       0.91      0.85      0.88        88
                   OTHER       0.41      0.57      0.48        93
           Price Targets       1.00      1.00      1.00        66
        Product Approval       1.00      0.69      0.82        72
       Products-Services       0.75      0.82      0.78        49
              Regulatory       0.65      0.78      0.71        85
    Stock price movement       0.65      0.87      0.75        76
         

/home/shadeform/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/shadeform/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/shadeform/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/shadeform